In [1]:
import csv
import xml.etree.ElementTree as ET

pops = []
with open("population_utf8.csv") as infile:
    rd = csv.reader(infile, delimiter=";")
    for line in rd:
        pops.append((line[0].split(";")[-0], int(line[2])))           
pops


[('Akaa', 17052),
 ('Alahärmä', 0),
 ('Alajärvi', 10171),
 ('Alastaro', 0),
 ('Alavieska', 2687),
 ('Alavus', 12103),
 ('Anjalankoski', 0),
 ('Artjärvi', 0),
 ('Asikkala', 8374),
 ('Askainen', 0),
 ('Askola', 5064),
 ('Aura', 3982),
 ('Brändö', 474),
 ('Dragsfjärd', 0),
 ('Eckerö', 932),
 ('Elimäki', 0),
 ('Eno', 0),
 ('Enonkoski', 1503),
 ('Enontekiö', 1890),
 ('Espoo', 265543),
 ('Eura', 12314),
 ('Eurajoki', 5954),
 ('Evijärvi', 2651),
 ('Finström', 2535),
 ('Forssa', 17521),
 ('Föglö', 568),
 ('Geta', 494),
 ('Haapajärvi', 7479),
 ('Haapavesi', 7175),
 ('Hailuoto', 997),
 ('Halikko', 0),
 ('Halsua', 1222),
 ('Hamina', 21061),
 ('Hammarland', 1531),
 ('Hankasalmi', 5307),
 ('Hanko', 9021),
 ('Harjavalta', 7366),
 ('Hartola', 3071),
 ('Hattula', 9738),
 ('Hauho', 0),
 ('Haukipudas', 0),
 ('Haukivuori', 0),
 ('Hausjärvi', 8815),
 ('Heinola', 19695),
 ('Heinävesi', 3638),
 ('Helsinki', 620715),
 ('Himanka', 0),
 ('Hirvensalmi', 2326),
 ('Hollola', 21892),
 ('Honkajoki', 1788),
 ('Houts

In [2]:
import pandas as pd

popDf = pd.DataFrame(pops)
popDf.columns = ['municipality', 'population']
popDf

,municipality,population
0,Akaa,17052
1,Alahärmä,0
2,Alajärvi,10171
3,Alastaro,0
4,Alavieska,2687
...,...,...
451,Ylöjärvi,32260
452,Ypäjä,2468
453,Äetsä,0
454,Ähtäri,6178


In [3]:
import numpy as np
spds = []
with open("results-2019-01-nopeustesti.csv",errors="ignore") as infile:
    rd = csv.reader(infile, delimiter=",")
    next(rd)
    for line in rd:
        spds.append((line[3].replace("ae", "ä").replace("AE","Ä").replace("oe","ö").replace("jön","joe"), int(line[4].replace('n/a', '0')), int(line[5].replace('n/a', '0'))))

spdsDf = pd.DataFrame(spds)
spdsDf.columns = ["municipality","DL avg", "UL avg"]
avgDf = spdsDf.groupby("municipality").mean().round().reset_index()


varDf = spdsDf.groupby("municipality").var().round().reset_index()
varDf.columns = ["municipality","DL var", "UL var"]


In [4]:
popAvg = pd.merge(left=avgDf, right = popDf, how="left", left_on="municipality", right_on="municipality")
popSpeeds = pd.merge(left=popAvg, right = varDf, how="left", left_on="municipality", right_on="municipality")
popSpeeds

,municipality,DL avg,UL avg,population,DL var,UL var
0,,29222.0,13399.0,NaN,9.960193e+08,2.351833e+08
1,Ahmovaara,48836.0,7353.0,NaN,6.642606e+08,1.852068e+07
2,Ahvio,17034.0,2427.0,NaN,1.720168e+08,9.766728e+06
3,Airaksela,44400.0,2867.0,NaN,8.839720e+08,7.648711e+06
4,Aitoo,13072.0,806.0,NaN,2.119709e+08,3.454420e+05
...,...,...,...,...,...,...
999,Ähtäri,28637.0,21013.0,6178.0,7.146832e+08,8.321656e+08
1000,Äijänneva,11579.0,6394.0,NaN,4.512437e+06,3.223110e+05
1001,Äitsaari,58534.0,35942.0,NaN,1.764261e+09,1.062507e+09
1002,Äystö,28686.0,17566.0,NaN,NaN,NaN


In [63]:
root = ET.Element('NetSpeedData')
for row in popSpeeds.iterrows():
    municipality = ET.SubElement(root, 'municipality')
    dlAvg = ET.SubElement(municipality, 'DLavg')
    ulAvg = ET.SubElement(municipality, 'ULavg')
    population = ET.SubElement(municipality, 'population')
    dlVar = ET.SubElement(municipality, 'DLvar')
    ulVar = ET.SubElement(municipality, 'ULvar')
    municipality.text = str(row[1]['municipality'])
    dlAvg.text = str(row[1]['DL avg'])
    ulAvg.text = str(row[1]['UL avg'])
    population.text = str(row[1]['population'])
    dlVar.text = str(row[1]['DL var'])
    ulVar.text = str(row[1]['UL var'])

xml = ET.tostring(root, encoding='utf8').decode('utf8')

with open('netspeeds.xml', 'w', encoding='utf8') as f:
    f.write(xml)

xml

"<?xml version='1.0' encoding='utf8'?>\n<NetSpeedData><municipality><DLavg>29222.0</DLavg><ULavg>13399.0</ULavg><population>nan</population><DLvar>996019324.0</DLvar><ULvar>235183282.0</ULvar></municipality><municipality>Ahmovaara<DLavg>48836.0</DLavg><ULavg>7353.0</ULavg><population>nan</population><DLvar>664260578.0</DLvar><ULvar>18520683.0</ULvar></municipality><municipality>Ahvio<DLavg>17034.0</DLavg><ULavg>2427.0</ULavg><population>nan</population><DLvar>172016823.0</DLvar><ULvar>9766728.0</ULvar></municipality><municipality>Airaksela<DLavg>44400.0</DLavg><ULavg>2867.0</ULavg><population>nan</population><DLvar>883972010.0</DLvar><ULvar>7648711.0</ULvar></municipality><municipality>Aitoo<DLavg>13072.0</DLavg><ULavg>806.0</ULavg><population>nan</population><DLvar>211970880.0</DLvar><ULvar>345442.0</ULvar></municipality><municipality>Akaa<DLavg>37536.0</DLavg><ULavg>14488.0</ULavg><population>17052.0</population><DLvar>2982846676.0</DLvar><ULvar>893404788.0</ULvar></municipality><mun

In [49]:
json = popSpeeds.to_json(orient = "records")

with open('netspeeds.json', 'w', encoding='utf8') as f:
    f.write(json)

json

'[{"municipality":"","DL avg":29222.0,"UL avg":13399.0,"population":null,"DL var":996019324.0,"UL var":235183282.0},{"municipality":"Ahmovaara","DL avg":48836.0,"UL avg":7353.0,"population":null,"DL var":664260578.0,"UL var":18520683.0},{"municipality":"Ahvio","DL avg":17034.0,"UL avg":2427.0,"population":null,"DL var":172016823.0,"UL var":9766728.0},{"municipality":"Airaksela","DL avg":44400.0,"UL avg":2867.0,"population":null,"DL var":883972010.0,"UL var":7648711.0},{"municipality":"Aitoo","DL avg":13072.0,"UL avg":806.0,"population":null,"DL var":211970880.0,"UL var":345442.0},{"municipality":"Akaa","DL avg":37536.0,"UL avg":14488.0,"population":17052.0,"DL var":2982846676.0,"UL var":893404788.0},{"municipality":"Ala-Valli","DL avg":12658.0,"UL avg":1079.0,"population":null,"DL var":480.0,"UL var":447458.0},{"municipality":"Alaj\\u00e4rvi","DL avg":45620.0,"UL avg":11304.0,"population":10171.0,"DL var":958465963.0,"UL var":77204423.0},{"municipality":"Alaniemi","DL avg":43977.0,"UL 